In [ ]:
%pip install tables

In [ ]:
import time
import argparse

import pandas as pd
import pytorch_lightning as pl
import torch
import torchtext


In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--train_batch_size", default=32, type=int)
parser.add_argument("--min_item_popularity", default=1, type=int)
parser.add_argument("--min_customer_purchase", default=1, type=int)
parser.add_argument("--LSTM_hidden_size", default=128, type=int)
parser.add_argument("--L1_size", default=128, type=int)
parser.add_argument("--emb_size", default=8, type=int)
parser.add_argument("--Embedding_per_art_feature", default=16, type=int)
parser.add_argument("--max_article_feature_levels", default=500, type=int)

In [ ]:
params = vars(parser.parse_args([]))

In [ ]:
class TxnDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        super().__init__()
        self.df = df.reset_index()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, n):
        row = self.df.iloc[n, :]
        item = row["item"]  # Raw

        wk = row["wk"]
        item = item

        return wk[:-1], item[:-1], item[1:], row["customer_id"]
unk_cid = []


def collate(l):
#     item_l = []
#     target_l = []
#     wk_l = []
#     cid_l = []

#     global unk_cid
#     for wk, item, target, cid in l:
#         cid_l.append(cid)
#         if wk.shape[0] == 0:
#             unk_cid.append(cid)
#             continue
#         item_l.append(item)
#         target_l.append(target)
#         wk_l.append(wk)

#    return wk_l, item_l, target_l, cid_l
    return list(map(list, zip(*l)))

In [ ]:
arts = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations//articles.csv", dtype=str)
arts.set_index("article_id", inplace=True)  # For faster indexing
arts = arts.drop(columns=["detail_desc"])

class ArticleEmbedding(pl.LightningModule):
    def __init__(self, params: dict = None):
        super().__init__()

        self.E = torch.nn.ModuleDict()
        self.V = dict()

        for col in arts.columns:
            levels = len(set(arts[col]))
            if levels > params["max_article_feature_levels"]:
                continue

            # No need for OOV. All features are fixed
            self.V[col] = torchtext.vocab.build_vocab_from_iterator(arts[col].apply(lambda x: [str(x)]),
                                                                    specials=["PAD"])

            self.E[col] = torch.nn.Embedding(num_embeddings=len(self.V[col]),
                                             embedding_dim=params["Embedding_per_art_feature"])

        self.embedding_dim = sum(e.embedding_dim for e in self.E.values())
        self.num_embeddings = len(arts) + 2

    def forward(self, items):
        rows = arts.loc[items]

        embs = []
        for col in self.E:
            embs.append(self.E[col](torch.tensor(
                self.V[col].lookup_indices(rows[col].tolist()), device=self.device
            )))

        embs = torch.cat(embs, dim=1)
        return embs

In [ ]:
class TxnModel(pl.LightningModule):
    def __init__(self, E_item, E_wk, V_item, params):
        super().__init__()

        self.E_item = E_item  # Input embedding
        self.E_wk = E_wk
        self.V_item = V_item  # For output layer

        # self.LSTM = torch.nn.LSTM(input_size=self.E_item.embedding_dim + self.E_wk.embedding_dim,
        #                           hidden_size=params["LSTM_hidden_size"], )

        # self.LayerNorm = torch.nn.LayerNorm(normalized_shape=params["LSTM_hidden_size"])
        #
        # self.L = torch.nn.Sequential(
        #     torch.nn.Linear(in_features=params["LSTM_hidden_size"],
        #                     out_features=params["L1_size"]),
        #     torch.nn.ReLU(),
        #     torch.nn.Linear(in_features=params["L1_size"],
        #                     out_features=E_item.num_embeddings),
        # )
        self.Attn = torch.nn.MultiheadAttention(embed_dim=self.E_item.embedding_dim + self.E_wk.embedding_dim,
                                                num_heads=4, batch_first=True)
        #self.LayerNorm = torch.nn.LayerNorm(normalized_shape=self.E_item.embedding_dim + self.E_wk.embedding_dim)

        self.L = torch.nn.Sequential(
            torch.nn.Linear(in_features=self.E_item.embedding_dim + self.E_wk.embedding_dim,
                            out_features=params["L1_size"]),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=params["L1_size"],
                            out_features=len(V_item)),
        )

        self.CE = torch.nn.CrossEntropyLoss(ignore_index=V_item.get_stoi()["PAD"])
        self.Dropout = torch.nn.Dropout(0)

    def get_embedding(self, wk, it):
        return torch.cat([self.E_item(it), self.E_wk(wk)], dim=1)

    def forward(self, wk, it):
        # item_pack = torch.nn.utils.rnn.pack_sequence([self.get_embedding(w, i)
        #                                               for w, i in zip(wk, it)],
        #                                              enforce_sorted=False)
        #
        # X, (hn, cn) = self.LSTM(item_pack)
        item_pad = torch.nn.utils.rnn.pad_sequence([self.get_embedding(w, i)
                                                    for w, i in zip(wk, it)], batch_first=True)
        b, l, e = item_pad.shape
        
        
        padding_mask = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(self.V_item.lookup_indices(i)).to(self.device) for i in it],
            batch_first=True) == 0  # The lookup is just a shortcut to get the padding mask
        
        # Disallow attention on future items and paddings
        X = self.Attn(item_pad, item_pad, item_pad,
                      attn_mask=torch.triu(torch.ones(l, l, device=self.device), diagonal=1),
                      key_padding_mask=padding_mask, need_weights=False)[0]

        O = self.L(X)

        return O

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    @staticmethod
    def acc12_metric(p: torch.tensor, tar: torch.tensor):
        v, idx = torch.topk(p, 12)
        return torch.sum(tar.unsqueeze(1) == idx) / tar.shape[0]

    def validation_step(self, batch, batch_idx):
        wk, it, tar, cid = batch

        tar = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(self.V_item.lookup_indices(t)).to(self.device) for t in tar],
            batch_first=True)

        p = self(wk, it)

        # Ignore PAD and UNK
        p = p[(tar != 0) & (tar != 1)]
        tar = tar[(tar != 0) & (tar != 1)]

        acc = torch.sum((torch.argmax(p, dim=1) == tar).int()) / tar.shape[0]

        self.log("val_loss", self.CE(p, tar), prog_bar=True, batch_size=1)
        self.log("val_acc", acc, prog_bar=True, batch_size=1)
        self.log("val_acc12", self.acc12_metric(p, tar), prog_bar=True, batch_size=1)

    def training_step(self, batch, batch_idx):
        wk, it, tar, cid = batch

        tar = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(self.V_item.lookup_indices(t)).to(self.device) for t in tar],
            batch_first=True)

        p = self(wk, it)
        loss = self.CE(p.reshape((-1, p.shape[-1])), tar.reshape((-1)))

        self.last_train_loss = loss.item()
        self.log("train_loss", loss, logger=True, on_step=True)

        return loss

In [ ]:
import pickle
AE = pickle.load(open("../input/hmmodel/AE.pickle", "rb"))
E_wk = pickle.load(open("../input/hmmodel/E_wk.pickle", "rb"))
V_item = pickle.load(open("../input/hmmodel/V_item.pickle", "rb"))
V_wk = pickle.load(open("../input/hmmodel/V_wk.pickle", "rb"))

In [ ]:
MM = TxnModel(AE, E_wk, V_item, params)

In [ ]:
MM.load_state_dict(torch.load("../input/hmmodel/model.ckpt")["state_dict"])
MM.cuda()
MM.eval();

In [ ]:
df_whole = pd.read_hdf("/kaggle/input/hmtxn/cust_week_seq_p4.h5", key="df")

In [ ]:
df_whole = df_whole.reset_index()

In [ ]:
df_whole["wk"] = df_whole["wk"].apply(lambda x: torch.tensor(V_wk.lookup_indices(x)))

In [ ]:
df_whole.head()

In [ ]:
import random
import numpy as np
all_items = list(V_item.get_stoi().keys())

In [ ]:
import time
t0 = time.time()

subm = []
failures = []

i = 0
total = len(df_whole) // 32
failure_cnt = 0

for rows in np.array_split(df_whole, len(df_whole) // 32):
    i += 1
    if i % 100 == 0:
        print(f"Batch {i} / {total}: {time.time()-t0:.2f}s")
    cid = rows["customer_id"]
    
    wk = [t.cuda() for t in rows["wk"]]
    it = rows["item"]
    try:
        p = MM.forward(wk, it)[:, -1, :]
        v, idx = torch.topk(p, 12, dim=1)
        products = [" ".join(V_item.lookup_tokens(i.flatten().tolist())) for i in idx.split(1, dim=0)]
    except KeyboardInterrupt:
        raise KeyboardInterrupt()
    except Exception as e:
        failures.append(cid)
        failure_cnt += 1
        print(f"Failure {failure_cnt}: batch {i}, {e}")
        products = [" ".join(random.choices(all_items, k=12)) for _ in range(len(cid))]
   
    subm.append(pd.DataFrame({"customer_id": cid, "prediction": products}))

In [ ]:
df_subm = pd.concat(subm)

In [ ]:
df_cust = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv")["customer_id"]

In [ ]:
df_join = pd.merge(df_subm, df_cust, on="customer_id", how="right")

In [ ]:
df_join["prediction"] = df_join["prediction"].apply(lambda s: s if not pd.isna(s) else " ".join(random.choices(all_items, k=12)))

In [ ]:
df_join.to_csv("./submission.csv", index=False)